Status: in progress

Remark: All the analysis is going to be aggregating all the data by hour, so doing it in that way, we can analyse all the years together and compare them

In [1]:
# packages 
import pandas as pd
import numpy as np

import missingno as msno 

from pymongo import MongoClient

import matplotlib.pyplot as plt
%matplotlib inline 

import seaborn as sns
sns.set_theme(style="ticks")


/var/folders/b0/69vxpwp54bj58xqzl4ynkwbw0000gn/T/ipykernel_17650/606176999.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
## connecting
db_name = "meteo_paranal" # meteo_paranal_test, meteo_paranal

client = MongoClient("mongodb://localhost:27017/")
db = client[db_name] # meteo_paranal_test
collection = db[db_name]

In [35]:
## utils (skip this cell)
numeric_col = [
   'Air Pressure at ground [hPa]',
   'Air Pressure Normalised [hPa]',
   'Air Temperature at 30m [C]',
   'Air Temperature at 2m [C]',
   'Air Temperature at ground [C]',
   'Air Temperature below VLT [C]',
   'Dew Temperature at 30m [C]',
   'Dew Temperature at 2m [C]',
   'Dew Temperature below VLT [C]',
   'Rain intensity below VLT [%]',
   'Relative Humidity at 30m [%]',
   'Relative Humidity at 2m [%]',
   'Relative Humidity below VLT [%]',
   'Wind Direction at 30m (0/360) [deg]',
   'Wind Direction at 30m (180/-180) [deg]',
   'Wind Direction at 10m (0/360) [deg]',
   'Wind Direction at 10m (180/-180) [deg]',
   'Wind Speed at 30m [m/s]',
   'Wind Speed at 10m [m/s]',
   'Wind Speed U at 20m [m/s]',
   'Wind Speed V at 20m [m/s]',
   'Wind Speed W at 20m [m/s]',
   'Air Pressure instantanous [hPa]',
   'Air Temperature instantanous at 30m [C]',
   'Air Temperature instantanous at 2m [C]',
   'Air Temperature instantanous at ground [C]',
   'Air Temperature instantaneous below VLT [C]',
   'Dew Temperature istantaneous at 30m [C]',
   'Dew Temperature instantanous at 2m [C]',
   'Dew Temperature instantanous below VLT [C]',
   'Humidity instantanous at 30m [%]',
   'Humidity instantanous at 2m [%]',
   'Humidity instantanous below VLT [%]',
   'Rain intensity instantanous [%]',
   'Wind Direction instantanous at 30m [deg]',
   'Wind Direction instantanous at 10m [deg]',
   'Wind Speed instantanous at 30m [m/s]',
   'Wind Speed instantanous at 10m [m/s]',
   'Wind Speed U instantanous at 20m [m/s]',
   'Wind Speed V instantanous at 20m [m/s]',
   'Wind Speed W instantanous at 20m [m/s]',
   'Particle (Large) at 20m [1/m^3]',
   'Particle (Large) at 10m [1/m^3]',
   'Particle (Small) at 20m [1/m^3]',
   'Particle (Small) at 10m [1/m^3]',
   'Particle (Large) Instantanous at 20m [1/m^3]',
   'Particle (Large) instantanous at 10m [1/m^3]',
   'Particle (Small) instantanous at 20m [1/m^3]',
   'Particle (Small) instantanous at 10m [1/m^3]',
 ]

def dict_to_datetime(x):
   return f"{x['year']}-{x['month']}-{x['day']} {x['hour']}:00:00"


def get_datetime_index(df):
    df['_id'] = pd.to_datetime(df['_id'].apply(dict_to_datetime))
    df = df.set_index(['_id'])
    df = df.sort_index(ascending=True)
    return df

def get_df(aggregated_expression):

    pipeline = [
        {"$group": {
                "_id": {
                    "year": {"$year": {"$dateFromString": {"dateString": "$Date time"}}},
                    "month": {"$month": {"$dateFromString": {"dateString": "$Date time"}}},
                    "day": {"$dayOfMonth": {"$dateFromString": {"dateString": "$Date time"}}},
                    "hour": {"$hour": {"$dateFromString": {"dateString": "$Date time"}}},
                }, 
                **aggregated_expression}},
        {"$sort": {"_id.year": 1, "_id.month": 1, "_id.day": 1, "_id.hour": 1} }
    ]

    result = list(collection.aggregate(pipeline))

    return pd.DataFrame(result)

## plot functions
def plot_and_save_hexbin(variable1,variable2):
    plt.clf()
    sns.jointplot(x=df[variable1], y=df[variable2], kind="hex", color="#4CB391")
    plt.xlabel(f'{variable1}')
    plt.ylabel(f'{variable2}')
    plt.title(f'Scatterplot {variable1} and {variable2}')
    plt.savefig(f"../plots/scatterplot/Scatterplot_{variable1.replace('/','')}_&&&_{variable2.replace('/','')}.png",dpi=100)

def plot_and_save_scatter(variable1,variable2):
    plt.clf()
    sns.jointplot(x=variable1,y=variable2,data=df)
    plt.xlabel(f'{variable1}')
    plt.ylabel(f'{variable2}')
    # plt.title(f'Scatterplot {variable1} and {variable2}')
    plt.savefig(f"../plots/scatterplot/Scatterplot_{variable1.replace('/','')}_&&&_{variable2.replace('/','')}.png",dpi = 100)

def plot_historic_histogram(variable):
    plt.clf()
    df[variable].plot(kind='hist',bins=100, title=f'Histogram for {variable}')
    plt.savefig(f"../plots/historic_histograms/histogram_{variable.replace('/','')}.png",dpi = 100)

def plot_density_by_year(variable):
    grouped_data = df.groupby(df.index.year)
    fig, axes = plt.subplots(nrows=len(grouped_data), figsize=(10, 5 * len(grouped_data)))
    for i, (year, group) in enumerate(grouped_data):
        sns.kdeplot(data=group[variable], ax=axes[i], fill=True, label=f'Year {year}')
        axes[i].set_title(f'Year {year}')
        # group[variable].plot(kind='hist', ax=axes[i], bins=10, title=f'Histogram for Year {year}')
        axes[i].set_xlim(df[variable].min(), df[variable].max())

    axes[-1].set_xlabel(variable)
    axes[-1].set_ylabel('Density')
    plt.tight_layout()
    plt.savefig(f"../plots/histograms_per_year/density_by_year_for_{variable.replace('/','')}.png",dpi = 100)

def plot_timeseries_err(variable):
    plt.clf()

    fig, ax = plt.subplots(figsize=(20, 7))
    plt.plot(df.index, df[variable], label = None)
    plt.fill_between (df_std.index, df[variable]-df_std[variable], df[variable]+df_std[variable], color='gray', alpha=0.3) # plot de la desviación
    plt.title(f'Historical time series for {variable}')
    plt.xlabel('Date')
    plt.ylabel(variable)
    # plt.gcf().set_size_inches(20, 7)
    for year in df.index.year.unique():
        plt.axvline(pd.to_datetime(f'{year}-01-01'), linestyle='--', color='black', alpha=0.5)
        plt.xticks(rotation=45, ha='right')

    plt.savefig(f"../plots/timeseries/timeseries_error_{variable.replace('/','')}.png", dpi=100)
    plt.show()

def plot_timeseries_subplots(variable):
    plt.clf()

    df['hourofyear'] = 24 * (df.index.dayofyear - 1) + df.index.hour
    
    unique_years = df.index.year.unique()
    num_years = len(unique_years)
    fig, axes = plt.subplots(nrows=num_years, ncols=1, figsize=(20, 7*num_years), sharex=True)

    for i, year in enumerate(unique_years):
        ax = axes[i] 
        year_data = df[df.index.year == year]
        # year_data_std = df_std[df_std.index==year]

        ax.plot(year_data.hourofyear, year_data[variable], label=None,linestyle='None', marker='o', markersize=2)
        ax.set_title(f'Historical time series for {variable} - Year {year}')
        ax.set_ylabel(variable)
        ax.tick_params(axis='x', rotation=45,)

    plt.xlabel('Date')
    plt.tight_layout()
    plt.savefig(f"../plots/timeseries_subplots/timeseries_subplots_{variable.replace('/','')}.png", dpi=100)



# correlation function
def get_principal_correlations(df_corr,n_display):

    upper_triangle_correlation = df_corr.where(np.triu(np.ones(df_corr.shape), k=1).astype(bool))
    top_positive_correlations = upper_triangle_correlation.unstack().sort_values(ascending=False)[:n_display]
    top_negative_correlations = upper_triangle_correlation.unstack().sort_values(ascending=True)[:n_display]

    rename_columns={'level_0':'feature1','level_1':'feature2',0:'Correlation'}

    less_correlated = upper_triangle_correlation.unstack().reset_index()
    less_correlated.rename(columns=rename_columns,inplace=True)
    less_correlated.Correlation=less_correlated.Correlation.apply(lambda x : abs(x))
    less_correlated= less_correlated.sort_values(by=['Correlation'],ascending=True)[less_correlated['Correlation']>0][:n_display]

    top_positive_correlations=top_positive_correlations.reset_index()
    top_positive_correlations.rename(columns=rename_columns,inplace=True)

    top_negative_correlations=top_negative_correlations.reset_index()
    top_negative_correlations.rename(columns=rename_columns,inplace=True)


    return top_negative_correlations,top_positive_correlations,less_correlated

Remark: that pipeline calculate the average per period only in the numeric values. Nan values are not be considered:
https://www.mongodb.com/docs/manual/reference/operator/aggregation/avg/

In [4]:
## uploading data

# defining what we want
mean_agg = {f: {"$avg": f"${f}"} for f in numeric_col}
std_agg = {f: {"$stdDevSamp": f"${f}"} for f in numeric_col}
# min_agg = {f: {"$min": f"${f}"} for f in numeric_col}
# max_agg = {f: {"$max": f"${f}"} for f in numeric_col}

# getting df
df_mean = get_df(mean_agg)
df_std = get_df(std_agg)
# df_min = get_df(min_agg)
# df_max = get_df(max_agg)

# getting datetime index
df_mean = get_datetime_index(df_mean)
df_std = get_datetime_index(df_std)
# df_min = get_datetime_index(df_min)
# df_max = get_datetime_index(df_max)


In [36]:
df=df_mean.copy() # just for save
print('DF shape: ',df.shape)
display(df.head(3))

DF shape:  (218645, 49)


,Air Pressure at ground [hPa],Air Pressure Normalised [hPa],Air Temperature at 30m [C],Air Temperature at 2m [C],Air Temperature at ground [C],Air Temperature below VLT [C],Dew Temperature at 30m [C],Dew Temperature at 2m [C],Dew Temperature below VLT [C],Rain intensity below VLT [%],...,Wind Speed V instantanous at 20m [m/s],Wind Speed W instantanous at 20m [m/s],Particle (Large) at 20m [1/m^3],Particle (Large) at 10m [1/m^3],Particle (Small) at 20m [1/m^3],Particle (Small) at 10m [1/m^3],Particle (Large) Instantanous at 20m [1/m^3],Particle (Large) instantanous at 10m [1/m^3],Particle (Small) instantanous at 20m [1/m^3],Particle (Small) instantanous at 10m [1/m^3]
_id,,,,,,,,,,,,,,,,,,,,,
1998-07-19 18:00:00,743.227800,1025.033800,11.8068,12.630600,15.082000,NaN,-11.900000,-11.934600,NaN,NaN,...,NaN,NaN,436.860000,213.840000,22346.740000,21574.120000,NaN,NaN,NaN,NaN
1998-07-19 19:00:00,742.884000,1024.588333,11.4845,12.151167,15.479833,NaN,-11.908000,-12.398667,NaN,NaN,...,NaN,NaN,352.816667,209.400000,17865.550000,15795.816667,NaN,NaN,NaN,NaN
1998-07-19 20:00:00,743.166333,1024.955333,11.2140,11.778667,15.683000,NaN,-12.614167,-12.911500,NaN,NaN,...,NaN,NaN,494.283333,199.633333,20182.683333,15636.966667,NaN,NaN,NaN,NaN


In [37]:
# df.info()
# df.describe()

In [54]:
## general values of nan values
plt.clf()
msno.matrix(df)
plt.title('Gráfico de Valores Nulos (blanco)')
plt.xlabel('Features')
plt.ylabel('Datos')
plt.gcf().set_size_inches(30, 25)
plt.savefig("../plots/nullplot.png",dpi = 100)

In [39]:
## These are variables with mostly nan values
nan_columns =   [ 
   'Particle (Large) at 20m [1/m^3]',
   'Particle (Large) at 10m [1/m^3]',
   'Particle (Small) at 20m [1/m^3]',
   'Particle (Small) at 10m [1/m^3]',
   'Particle (Large) Instantanous at 20m [1/m^3]',
   'Particle (Large) instantanous at 10m [1/m^3]',
   'Particle (Small) instantanous at 20m [1/m^3]',
   'Particle (Small) instantanous at 10m [1/m^3]',]

In [57]:
# plotting by dates
date_i = None
date_f =  '2007-01-10 12:00:00' #'2020-08-25 12:00:00'
plt.clf()
msno.matrix(df.loc[date_i:date_f])
plt.title('Gráfico de Valores Nulos (blanco)')
plt.xlabel('Features')
plt.ylabel('Datos')
plt.savefig("../plots/nullsplot_date.png",dpi = 100)


In [41]:
# some variables where it started to appear null values
extra_cols = ['Air Pressure instantanous [hPa]',
 'Air Temperature instantanous at 30m [C]',
 'Air Temperature instantanous at 2m [C]',
 'Air Temperature instantanous at ground [C]',
 'Air Temperature instantaneous below VLT [C]',
 'Dew Temperature istantaneous at 30m [C]',
 'Dew Temperature instantanous at 2m [C]',]

In [42]:
## How it is looks nan values in extra_cols
df.loc['2006-12-31 10:00:00':'2006-12-31 14:00:00',extra_cols]
### 2006-12-31 12:00:00

,Air Pressure instantanous [hPa],Air Temperature instantanous at 30m [C],Air Temperature instantanous at 2m [C],Air Temperature instantanous at ground [C],Air Temperature instantaneous below VLT [C],Dew Temperature istantaneous at 30m [C],Dew Temperature instantanous at 2m [C]
_id,,,,,,,
2006-12-31 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-12-31 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-12-31 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-12-31 13:00:00,744.418333,16.850833,17.338333,19.563167,18.654500,-6.141667,-6.199500
2006-12-31 14:00:00,744.453333,17.144667,17.774833,19.443667,19.147667,-5.721667,-5.789333


* <= 2006-12-31 12:00:00 there is no data for some columns, these are for example in the variable "extra_cols".
Also if you check paranal samples in the endpoint, you can see there is nothing before to the corresponding variants of those columns. You can check these dates "2006-12-31..2007-01-01" selecting all fields. http://archive.eso.org/wdb/wdb/asm/meteo_paranal/form


In [43]:
# dropping what we won't use
df=df_mean.copy()
df = df.loc['2006-12-31 13:00:00':]
df = df.drop(columns=nan_columns,axis=1)
numeric_col = list(df.columns)

## Correlations

In [44]:
## getting principal correlations in excel format, located in corraltions/
top_negative_correlations,top_positive_correlations,less_correlated = get_principal_correlations(df.corr(),n_display=150)

/var/folders/b0/69vxpwp54bj58xqzl4ynkwbw0000gn/T/ipykernel_17650/1753898554.py:172: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  less_correlated= less_correlated.sort_values(by=['Correlation'],ascending=True)[less_correlated['Correlation']>0][:n_display]


In [45]:
top_negative_correlations.to_excel('correlations/top_negative_correlations.xlsx',index=False)
top_positive_correlations.to_excel('correlations/top_positive_correlations.xlsx' ,index=False)
less_correlated.to_excel('correlations/less_correlated.xlsx',index=False)

# Plot Analysis

## Historic Histogram

* Folder: ../plots/historic_histograms/

In [46]:
for variable in numeric_col:
    plot_historic_histogram(variable) # this function save the plot

In [47]:
### REMARK
# # Most stranges distributions:
# # 'Wind Direction at 30m (0/360) [deg]',----
# # 'Wind Direction at 30m (180/-180) [deg]',----
# # 'Wind Direction at 10m (0/360) [deg]',----
# # 'Wind Direction at 10m (180/-180) [deg]',----
# # 'Wind Direction instantanous at 30m [deg]',----
# # 'Wind Direction instantanous at 10m [deg]',----

# # Distributions which small range values:
# # 'Particle (Large) at 20m [1/m^3]',++
# # 'Particle (Large) at 10m [1/m^3]',++
# # 'Particle (Small) at 20m [1/m^3]',++
# # 'Particle (Small) at 10m [1/m^3]',++
# # 'Particle (Large) Instantanous at 20m [1/m^3]',++
# # 'Particle (Large) instantanous at 10m [1/m^3]',++
# # 'Particle (Small) instantanous at 20m [1/m^3]',++
# # 'Particle (Small) instantanous at 10m [1/m^3]'++
# # 'Wind Speed W at 20m [m/s]',++
# # 'Wind Speed W instantanous at 20m [m/s]',++

# # rest of them follow a distribution normal-like or log-normal

In [48]:
df_mean['Particle (Small) instantanous at 10m [1/m^3]'].describe() ## maybe for all of these features (refering to particle...) do a zoom in

count    1.455200e+04
mean     2.003910e+04
std      8.064640e+04
min      0.000000e+00
25%      9.333333e+00
50%      2.171667e+01
75%      1.916165e+03
max      6.073179e+06
Name: Particle (Small) instantanous at 10m [1/m^3], dtype: float64

## Histogram by year
* Folder: ../plots/histograms_per_year/

In [49]:
for variable in numeric_col:
    plot_density_by_year(variable) # save the plot

/var/folders/b0/69vxpwp54bj58xqzl4ynkwbw0000gn/T/ipykernel_17650/1753898554.py:108: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  sns.kdeplot(data=group[variable], ax=axes[i], fill=True, label=f'Year {year}')
/var/folders/b0/69vxpwp54bj58xqzl4ynkwbw0000gn/T/ipykernel_17650/1753898554.py:108: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  sns.kdeplot(data=group[variable], ax=axes[i], fill=True, label=f'Year {year}')


In [50]:
### REMARKS
# # Distributions which varies much per year
# # Air Temperature at ground [C]
# # Air Temperature below VLT [C]
# # Air Temperature instantaneous below VLT [C]',
# # Air Temperature instantanous at 2m [C]',
# # Air Temperature instantanous at 30m [C]
# # Air Temperature instantanous at ground [C]

# # 'Dew Temperature at 30m [C]',--
# # 'Dew Temperature at 2m [C]',--
# # 'Dew Temperature below VLT [C]',--
# # 'Dew Temperature istantaneous at 30m [C]',--
# # 'Dew Temperature instantanous at 2m [C]',--
# # 'Dew Temperature instantanous below VLT [C]',--
# # 'Humidity instantanous below VLT [%]'

# # 'Wind Direction at 30m (0/360) [deg]',----
# # 'Wind Direction at 30m (180/-180) [deg]',----
# # 'Wind Direction at 10m (0/360) [deg]',----
# # 'Wind Direction at 10m (180/-180) [deg]',----

## Scatterplot

In [51]:
# Here I selected most interesting correlations I saw
# most correlated
pair1 = ('Humidity instantanous at 2m [%]',	'Dew Temperature instantanous at 2m [C]')
pair2 = ('Rain intensity instantanous [%]',	'Humidity instantanous at 2m [%]')
pair3 = ('Rain intensity instantanous [%]',	'Humidity instantanous at 30m [%]')

# less correlated
pair4 = ('Wind Speed V at 20m [m/s]',	'Air Temperature below VLT [C]')

pairs = (pair1,pair2,pair3,pair4)
for pair in pairs:
    plot_and_save_scatter(pair[0],pair[1])

* pair1 interesting

## Historical Timeseries error

In [52]:
for variable in numeric_col:
    plot_timeseries_err(variable) # saving plots

/var/folders/b0/69vxpwp54bj58xqzl4ynkwbw0000gn/T/ipykernel_17650/1753898554.py:133: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/b0/69vxpwp54bj58xqzl4ynkwbw0000gn/T/ipykernel_17650/1753898554.py:133: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/b0/69vxpwp54bj58xqzl4ynkwbw0000gn/T/ipykernel_17650/1753898554.py:133: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/b0/69vxpwp54bj58xqzl4ynkwbw0000gn/T/ipykernel_17650/1753898554.py:133: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/b0/69vxpwp54bj58xqzl4ynkwbw0000gn/T/ipykernel_17650/1753898554.py:133: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/b0/69vxpwp54bj58xqzl4ynkwbw0000gn/T/ipykernel_17650/1753898554.py:133: UserWarning: FigureCanvasAgg is non-interactive, and thus can

## time series subplots

In [53]:
for variable in numeric_col:
    plot_timeseries_subplots(variable)

Variables which shows frequency insteresting in the first touch
* Air Temperature at ground [C] 
* Air Temperature instantaneous below VLT [C]
* All Dew temperature
* humidity: tends to get high in first semester of every year
*  "wind directions" variables shows a lot of variance over all year without any pattern 

### Next steps
* see variables periodicity by season, night/day
* see variables at differents levels (i.e. ground, 2m, 10m, 20m, etc)
* what we find if we apply clusterization
* do the same but with different aggregation, here we did it with 1 hour, doing it with 15 min would be more accurate